In [5]:
import folium
import pandas as pd
import re

from operator import itemgetter
from heapq import nsmallest
from heapq import nlargest

import openrouteservice as ors

client = ors.Client(key='5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf')

In [3]:
df = pd.read_csv('../data/curated/property_full.csv')

In [18]:
# extract information from the top 10 most expensive properties and top 10 cheapest properties
# and plot their how they differ in distances from CBD, closest school, closet shopping centre and cloest train station.

# find the 10 most expensive properties 

index = list(range(len(df)))
pairs = zip(index, df['cost_text'])
result = nlargest(10, pairs, key=itemgetter(1))
index_list = [i[0] for i in result]

# store the all the information from the 10 most expensive properties 
expensive_10 = [df.iloc[i] for i in index_list]

# find the 10 cheapest properties 
index = list(range(len(df)))
pairs = zip(index, df['cost_text'])
result = nsmallest(10, pairs, key=itemgetter(1))
index_list = [i[0] for i in result]

# store the all the information from the 10 cheapest properties 
cheapest_10 = [df.iloc[i] for i in index_list]


In [28]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14767 entries, 0 to 14766
Data columns (total 70 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Unnamed: 0                                14767 non-null  int64  
 1   Unnamed: 0_x                              14767 non-null  int64  
 2   Unnamed: 0.1                              14767 non-null  object 
 3   name                                      14767 non-null  object 
 4   cost_text                                 14767 non-null  int64  
 5   coordinates                               14767 non-null  object 
 6   rooms                                     14767 non-null  int64  
 7   baths                                     14767 non-null  int64  
 8   parking                                   14767 non-null  int64  
 9   driving_time_to_school                    14767 non-null  float64
 10  distance_to_school                

In [36]:
# plot the routes from the extracted properties to the location of Melbourne CBD
# label each item with their rent cost
m = folium.Map([-38.013995, 145.264296], zoom_start=9)

cbd = [144.946457,-37.840935]
folium.Marker(location=(cbd[1], cbd[0]), icon=folium.Icon(icon='building', color='blue', prefix='fa')).add_to(m)

for property in expensive_10:
    

    lat = float(re.findall(r'\[(-\d+.\d+)',str(property['coordinates']))[0])
    long = float(re.findall(r'(\d+.\d+)\]',str(property['coordinates']))[0])

    folium.Marker(location=(lat,long), icon=folium.Icon(icon='home', color='red', prefix='fa'), popup=property['cost_text']).add_to(m)

    coordinates = [cbd, [long,lat]]
    
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

for property in cheapest_10:
    
    lat = float(re.findall(r'\[(-\d+.\d+)',str(property['coordinates']))[0])
    long = float(re.findall(r'(\d+.\d+)\]',str(property['coordinates']))[0])

    folium.Marker(location=(lat,long), icon=folium.Icon(icon='home', color='purple', prefix='fa'), popup=property['cost_text']).add_to(m)

    coordinates = [cbd, [long,lat]]
    
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
                           
m.save('../plots/cbd_route.html')

m

In [38]:
m = folium.Map([-38.013995, 145.264296], zoom_start=9)

for property in expensive_10:

    lat_pro = float(re.findall(r'\[(-\d+.\d+)',str(property['coordinates']))[0])
    long_pro = float(re.findall(r'(\d+.\d+)\]',str(property['coordinates']))[0])

    folium.Marker(location=(lat_pro,long_pro), icon=folium.Icon(icon='home', color='red', prefix='fa')).add_to(m)

    long_school = float(re.findall(r'\((\d+.\d+)',str(property['closest_school_coor']))[0])
    lat_school = float(re.findall(r'(-\d+.\d+)\)',str(property['closest_school_coor']))[0])

    folium.Marker(location=(lat_school,long_school), icon=folium.Icon(icon='graduation-cap', color='blue', prefix='fa')).add_to(m)

    coordinates = [[long_school,lat_school], [long_pro,lat_pro]]
    
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

for property in cheapest_10:

    lat_pro = float(re.findall(r'\[(-\d+.\d+)',str(property['coordinates']))[0])
    long_pro = float(re.findall(r'(\d+.\d+)\]',str(property['coordinates']))[0])

    folium.Marker(location=(lat_pro,long_pro), icon=folium.Icon(icon='home', color='purple', prefix='fa')).add_to(m)

    long_school = float(re.findall(r'\((\d+.\d+)',str(property['closest_school_coor']))[0])
    lat_school = float(re.findall(r'(-\d+.\d+)\)',str(property['closest_school_coor']))[0])

    folium.Marker(location=(lat_school,long_school), icon=folium.Icon(icon='graduation-cap', color='blue', prefix='fa')).add_to(m)

    coordinates = [[long_school,lat_school], [long_pro,lat_pro]]
    
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

m.save('../plots/school_route.html')

m

In [40]:
m = folium.Map([-38.013995, 145.264296], zoom_start=9)

for property in expensive_10:

    lat_pro = float(re.findall(r'\[(-\d+.\d+)',str(property['coordinates']))[0])
    long_pro = float(re.findall(r'(\d+.\d+)\]',str(property['coordinates']))[0])

    folium.Marker(location=(lat_pro,long_pro), icon=folium.Icon(icon='home', color='red', prefix='fa')).add_to(m)

    long_station = float(re.findall(r'\((\d+.\d+)',str(property['closest_station_coor']))[0])
    lat_station = float(re.findall(r'(-\d+.\d+)\)',str(property['closest_station_coor']))[0])

    folium.Marker(location=(lat_station,long_station), icon=folium.Icon(icon='train', color='blue', prefix='fa')).add_to(m)

    coordinates = [[long_station,lat_station], [long_pro,lat_pro]]
    
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

for property in cheapest_10:

    lat_pro = float(re.findall(r'\[(-\d+.\d+)',str(property['coordinates']))[0])
    long_pro = float(re.findall(r'(\d+.\d+)\]',str(property['coordinates']))[0])

    folium.Marker(location=(lat_pro,long_pro), icon=folium.Icon(icon='home', color='purple', prefix='fa')).add_to(m)

    long_station = float(re.findall(r'\((\d+.\d+)',str(property['closest_station_coor']))[0])
    lat_station = float(re.findall(r'(-\d+.\d+)\)',str(property['closest_station_coor']))[0])

    folium.Marker(location=(lat_station,long_station), icon=folium.Icon(icon='train', color='blue', prefix='fa')).add_to(m)

    coordinates = [[long_station,lat_station], [long_pro,lat_pro]]
    
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

m.save('../plots/station_route.html')

m

In [41]:
m = folium.Map([-38.013995, 145.264296], zoom_start=9)

for property in expensive_10:

    lat_pro = float(re.findall(r'\[(-\d+.\d+)',str(property['coordinates']))[0])
    long_pro = float(re.findall(r'(\d+.\d+)\]',str(property['coordinates']))[0])

    folium.Marker(location=(lat_pro,long_pro), icon=folium.Icon(icon='home', color='red', prefix='fa')).add_to(m)

    long_sc = float(re.findall(r'\((\d+.\d+)',str(property['closest_sc_coor']))[0])
    lat_sc = float(re.findall(r'(-\d+.\d+)\)',str(property['closest_sc_coor']))[0])

    folium.Marker(location=(lat_sc,long_sc), icon=folium.Icon(icon='shopping-bag', color='blue', prefix='fa')).add_to(m)

    coordinates = [[long_sc,lat_sc], [long_pro,lat_pro]]
    
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

for property in cheapest_10:

    lat_pro = float(re.findall(r'\[(-\d+.\d+)',str(property['coordinates']))[0])
    long_pro = float(re.findall(r'(\d+.\d+)\]',str(property['coordinates']))[0])

    folium.Marker(location=(lat_pro,long_pro), icon=folium.Icon(icon='home', color='purple', prefix='fa')).add_to(m)

    long_sc = float(re.findall(r'\((\d+.\d+)',str(property['closest_sc_coor']))[0])
    lat_sc = float(re.findall(r'(-\d+.\d+)\)',str(property['closest_sc_coor']))[0])

    folium.Marker(location=(lat_sc,long_sc), icon=folium.Icon(icon='shopping-bag', color='blue', prefix='fa')).add_to(m)

    coordinates = [[long_sc,lat_sc], [long_pro,lat_pro]]
    
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

m.save('../plots/shop_route.html')

m